In [1]:
#Importing libraries
import os
import time
import json
import numpy as np
import torch
import pickle
from torch.autograd import Variable
import cv2
from matplotlib import pyplot as plt
from cnn import SegmentationModel as net

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
print(device)

cuda


In [3]:
def colorir(cat):
    h, w = cat.shape[:2]
    msk = np.zeros((h,w,3), dtype = 'uint8')
    msk[cat == 0] = [0,0,0]
    msk[cat == 1] = [0,128,0]
    msk[cat == 2] = [0,0,255]
    return(msk)

In [4]:
class MyArgs():
    def __init__(self):
        self.model = 'ESPNet'
        self.data_dir = './'
        self.scaleIn =  1
        self.max_epochs =  201
        self.num_workers =  4
        self.batch_size =  12
        self.step_loss =  100
        self.lr =  0.0005
        self.savedir =  './data/'
        self.classes =  4
        self.cached_data_file =  'stats.p'
        self.logFile =  'trainValLog.txt'
        self.pretrained =  './data/model_201.pth'
        self.s =  0.5
        self.resume =  True
        self.resumeLoc =  './data/checkpoint.pth.tar'
        self.onGPU =  True
        
    def __repr__(self):
        return str(self.__dict__)
    
    def __str__(self):
        return str(self.__dict__)

In [5]:
args = MyArgs()
args

{'model': 'ESPNet', 'data_dir': './', 'scaleIn': 1, 'max_epochs': 201, 'num_workers': 4, 'batch_size': 12, 'step_loss': 100, 'lr': 0.0005, 'savedir': './data/', 'classes': 4, 'cached_data_file': 'stats.p', 'logFile': 'trainValLog.txt', 'pretrained': './data/model_201.pth', 's': 0.5, 'resume': True, 'resumeLoc': './data/checkpoint.pth.tar', 'onGPU': True}

In [6]:
def calc_tiles(t, i, m):
    n = int(np.round(t/(i-m)))
    if n < 2:
        mc = co = np.array([[0],[t]])
    else:
        tt = np.repeat(np.array([[-.5,.5]]), n - 2, axis= 0)
        tt = np.concatenate([np.array([[0, 1]]), tt, np.array([[-1, 0]])])
        tc = t/n * np.stack([np.arange(n), 1 + np.arange(n)])
        ma = i - (tc[1] - tc[0])
        mc = np.int0(tc + tt.T * ma)
        co = np.int0(np.array([[0],[i]]) - tt.T * ma)
    return(mc, co)

def calc_bb(tw, th, iw, ih, m):
    xc, xco = calc_tiles(tw, iw, m)
    yc, yco = calc_tiles(th, ih, m)
    return([xc, yc, xco, yco])

In [7]:
def get_coords(contours, minArea, maxArea):
    pts = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area >  minArea and area < maxArea:
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            pts.append([cx,cy,area])
    return(np.array(pts))

In [8]:
def get_pts(cat, catn, minArea, maxArea, scale = 1):
    plant_msk = (cat == catn).astype('uint8')
    im_tmp, contours, hierarchy = cv2.findContours(plant_msk, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    pts = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area >  minArea and area < maxArea:
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            pts.append([cx,cy,area])
    return(np.int0(scale*np.array(pts)).astype(int))

In [9]:
def get_cat(img):
    start_time = time.time()
    
    img = img.astype(np.float32)
    img -= rgb_mean
    img /= rgb_std

    total_width, total_height = img.shape[:2]
    xc, yc, xco, yco = calc_bb(total_width, total_height, input_height, input_width, 40)

    time_taken = time.time() - start_time
    print('PreProc time: %.2f' % time_taken)

    preds_l = []
    for yi in range(yc.shape[1]):
        preds_x = []
        for xi in range(xc.shape[1]):
            preds = np.zeros((classes, xco[1,xi] - xco[0,xi],  yco[1,yi] - yco[0,yi]))
            im = img[xc[0,xi]:xc[1,xi],yc[0,yi]:yc[1,yi]]
            if im.mean() > -5:
                im = im.reshape(np.insert(im.shape, 0, 1))
                im = np.moveaxis(im, 3, 1)
                img_tensor = torch.from_numpy(im)
                img_variable = Variable(img_tensor)
                img_variable = img_variable.to(device)
                img_out, states = model(img_variable)
                preds = img_out.cpu().data.numpy()
                preds = preds[0,:,xco[0,xi]:xco[1,xi],yco[0,yi]:yco[1,yi]]
            preds_x.append(preds)

        preds_l.append(np.concatenate(preds_x, 1))

    time_taken = time.time() - start_time - time_taken
    print('Prediction time: %.2f' % time_taken)

    pred = np.concatenate(preds_l, 2)
    cat = np.argmax(pred, 0)

    time_taken = time.time() - start_time
    print('Total time: %.2f' % time_taken)
    return cat

In [10]:
def coords_to_json(coords, filename):    
    regions = []
    for cat_name in coords.keys():
        region_attri = {"Planta":cat_name}
        for pt in coords.get(cat_name):
            if cat_name == 'Crop Row':
                pt = {"name":"polyline","all_points_x":pt[:,0].tolist(),"all_points_y":pt[:,1].tolist()}
            else:        
                pt = {"name":"point","cx":int(pt[0]),"cy":int(pt[1])} 
            regions.append({'region_attributes':region_attri, 'shape_attributes':pt})

    file_size = int(os.path.getsize(filename))
    img_name = os.path.basename(filename)
    file_attri = {'filename': img_name,
                  'size' : file_size,
                  'regions' : regions,
                  'file_attributes': {}}
    via_img_metadata = {img_name + str(file_size):file_attri}
    return(via_img_metadata)

In [11]:
img_dir = '../../Datasets/CottonGP/20190108_LR/imgs'
img_ext = '.jpg'
imgs_name = [os.path.join(img_dir, i) for i in sorted(os.listdir(img_dir)) if i.endswith(img_ext)]
num_im = len(imgs_name)
krt = os.listdir(img_dir)
imgs_name

['../../Datasets/CottonGP/20190108_LR/imgs\\org_0a74acbe5da2ab33_1546965846000[1].jpg',
 '../../Datasets/CottonGP/20190108_LR/imgs\\org_34f1526fba9584cd_1546965712000[1].jpg',
 '../../Datasets/CottonGP/20190108_LR/imgs\\org_4b75406923f4f44c_1546966104000[1].jpg',
 '../../Datasets/CottonGP/20190108_LR/imgs\\org_62365da904094aa1_1546966756000[1].jpg',
 '../../Datasets/CottonGP/20190108_LR/imgs\\org_715b9f7643ed2c8e_1546966082000[1].jpg',
 '../../Datasets/CottonGP/20190108_LR/imgs\\org_a102fbdcb162488c_1546966736000[1].jpg',
 '../../Datasets/CottonGP/20190108_LR/imgs\\org_bcad5ec4e9a04b38_1546966490000[1].jpg',
 '../../Datasets/CottonGP/20190108_LR/imgs\\org_d7dd54101b32f9ad_1546966310000[1].jpg']

In [12]:
model = net.EESPNet_Seg(args.classes, s=args.s)
model_weight_file = './data/model_21.pth'
model.load_state_dict(torch.load(model_weight_file))
model = model.cuda()
# set to evaluation mode
model.eval()

data = pickle.load(open(args.cached_data_file, "rb"))

print('Modelo Ok')

Modelo Ok


In [13]:
#total = []

# gloabl mean and std values
rgb_mean = data['mean']
rgb_std = data['std']
n_classes = len(data['classWeights'])
new_ext = '.png'
classes = 4

input_height = 512
input_width = 512

all_json = {}
for img_name in imgs_name:
    img = cv2.imread(img_name)
    cat = get_cat(img)
    pred = colorir(cat)
    
    cotton_coords = get_pts(cat, 2, 1, 200)
    print(len(cotton_coords))

    coords_json = {'Cotton':cotton_coords}
    coords_json = coords_to_json(coords_json, img_name)
    all_json.update(coords_json)
    

PreProc time: 0.08
Prediction time: 2.31
Total time: 2.49
136
PreProc time: 0.07
Prediction time: 0.50
Total time: 0.69
42
PreProc time: 0.08
Prediction time: 0.49
Total time: 0.65
5
PreProc time: 0.14
Prediction time: 0.54
Total time: 0.80
27
PreProc time: 0.07
Prediction time: 0.47
Total time: 0.65
249
PreProc time: 0.08
Prediction time: 0.50
Total time: 0.70
132
PreProc time: 0.08
Prediction time: 0.49
Total time: 0.68
428
PreProc time: 0.08
Prediction time: 0.54
Total time: 0.76
358


In [14]:
via_settings = {'ui': {'annotation_editor_height': 25,
   'annotation_editor_fontsize': 0.8,
   'leftsidebar_width': 18,
   'image_grid': {'img_height': 80,
    'rshape_fill': 'none',
    'rshape_fill_opacity': 0.3,
    'rshape_stroke': 'yellow',
    'rshape_stroke_width': 2,
    'show_region_shape': True,
    'show_image_policy': 'all'},
   'image': {'region_label': 'region_id',
    'region_label_font': '10px Sans',
    'on_image_annotation_editor_placement': 'NEAR_REGION'}},
  'core': {'buffer_size': 18, 'filepath': {}, 'default_filepath': img_dir},
  'project': {'name': 'via_project_SmartAgri'}}

via_attributes = {'region': {'Planta': {'type': 'dropdown',
    'description': '',
    'options': {'Linha': '',
     'Milho': '',
     'Soja': '',
     'Cotton': ''},
    'default_options': {}}},
  'file': {}}

json_data = {'_via_img_metadata': all_json,
             '_via_settings': via_settings,
             '_via_attributes': via_attributes}

with open('./data/via_cotton.json', 'w') as jsonfile:
    json.dump(json_data, jsonfile)